In [1]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate, SystemMessagePromptTemplate,ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.agents import load_tools,initialize_agent,AgentType,create_react_agent,AgentExecutor
f = open('../api_key.txt')
api_key = f.read()
llm = ChatOpenAI(openai_api_key=api_key,temperature=0) #Recomendable temperatura a 0 para que el LLM no sea muy creativo, vamos a tener muchas herramientas a nuestra disposición y queremos que sea más determinista

In [2]:
import mysql.connector #pip install mysql-connector-python

In [3]:
f = open('./password_sql.txt')
pass_sql = f.read()
# Configuración de la conexión a la base de datos
config = {
    'user': 'root',       
    'password': pass_sql, 
    'host': '127.0.0.1',         
    'database': 'world'          
}

In [4]:
# Conectar a la base de datos
conn = mysql.connector.connect(**config)
cursor = conn.cursor()

In [5]:
# Definir la consulta manualmente: tengo una base de datos mysql en mi computadora local denominada "world" y una tabla "Country" 
#sobre la que quiero hacer la suma de la población en la columna "Population" para el continente Asia (columna "Continent")
query = """
    SELECT SUM(Population)
    FROM Country
    WHERE Continent = 'Asia';
    """

# Ejecutar la consulta
cursor.execute(query)
result = cursor.fetchone()

In [6]:
suma_poblacion = result[0] if result[0] is not None else 0
print(f"La suma de la población del continente Asia es: {suma_poblacion}")

La suma de la población del continente Asia es: 3705025700


In [7]:
from langchain_community.agent_toolkits import create_sql_agent
from langchain.sql_database import SQLDatabase

In [8]:
# Crear una cadena de conexión a la base de datos MySQL
connection_string = f"mysql+mysqlconnector://{config['user']}:{config['password']}@{config['host']}/{config['database']}"

# Crear una instancia de la base de datos SQL
db = SQLDatabase.from_uri(connection_string)

In [9]:
agent = create_sql_agent(
    llm,
    db=db,
    verbose=True
)

In [10]:
agent.invoke("Dime la población total de Asia")



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: city, country, countrylanguageI should query the country table to get the population of Asia.
Action: sql_db_schema
Action Input: country
CREATE TABLE country (
	`Code` CHAR(3) NOT NULL DEFAULT '', 
	`Name` CHAR(52) NOT NULL DEFAULT '', 
	`Continent` ENUM('Asia','Europe','North America','Africa','Oceania','Antarctica','South America') NOT NULL DEFAULT 'Asia', 
	`Region` CHAR(26) NOT NULL DEFAULT '', 
	`SurfaceArea` DECIMAL(10, 2) NOT NULL DEFAULT '0.00', 
	`IndepYear` SMALLINT, 
	`Population` INTEGER NOT NULL DEFAULT '0', 
	`LifeExpectancy` DECIMAL(3, 1), 
	`GNP` DECIMAL(10, 2), 
	`GNPOld` DECIMAL(10, 2), 
	`LocalName` CHAR(45) NOT NULL DEFAULT '', 
	`GovernmentForm` CHAR(45) NOT NULL DEFAULT '', 
	`HeadOfState` CHAR(60), 
	`Capital` INTEGER, 
	`Code2` CHAR(2) NOT NULL DEFAULT '', 
	PRIMARY KEY (`Code`)
)COLLATE utf8mb4_0900_ai_ci DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB

/*
3 rows from country table:
Co

{'input': 'Dime la población total de Asia', 'output': '3,705,025,700'}

In [11]:
result = agent.invoke("Dime el promedio de la esperanza de vida por cada una de las regiones ordenadas de mayor a menor")



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: city, country, countrylanguageI should query the schema of the country table to see if it contains information about regions and life expectancy.
Action: sql_db_schema
Action Input: country
CREATE TABLE country (
	`Code` CHAR(3) NOT NULL DEFAULT '', 
	`Name` CHAR(52) NOT NULL DEFAULT '', 
	`Continent` ENUM('Asia','Europe','North America','Africa','Oceania','Antarctica','South America') NOT NULL DEFAULT 'Asia', 
	`Region` CHAR(26) NOT NULL DEFAULT '', 
	`SurfaceArea` DECIMAL(10, 2) NOT NULL DEFAULT '0.00', 
	`IndepYear` SMALLINT, 
	`Population` INTEGER NOT NULL DEFAULT '0', 
	`LifeExpectancy` DECIMAL(3, 1), 
	`GNP` DECIMAL(10, 2), 
	`GNPOld` DECIMAL(10, 2), 
	`LocalName` CHAR(45) NOT NULL DEFAULT '', 
	`GovernmentForm` CHAR(45) NOT NULL DEFAULT '', 
	`HeadOfState` CHAR(60), 
	`Capital` INTEGER, 
	`Code2` CHAR(2) NOT NULL DEFAULT '', 
	PRIMARY KEY (`Code`)
)COLLATE utf8mb4_0900_ai_ci DEFAULT CHARSET=utf

In [12]:
# Mostrar el resultado
print(result["output"])

El promedio de la esperanza de vida por cada una de las regiones ordenadas de mayor a menor es:
1. Australia and New Zealand - 78.80
2. Nordic Countries - 78.33
3. Western Europe - 78.26
4. British Islands - 77.25
5. Southern Europe - 76.53
6. North America - 75.82
7. Eastern Asia - 75.25
8. Caribbean - 73.06
9. Central America - 71.03
10. South America - 70.95


In [ ]:
# Para utilizar few-shoots para las consultas SQL: https://python.langchain.com/v0.1/docs/use_cases/sql/agents/